<a href="https://colab.research.google.com/github/sholaremu/sholaremu/blob/main/Feature_engineering_on_image_data_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import skimage
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io

%matplotlib inline

In [ ]:
cat = io.imread('cat.jpg')
dog = io.imread('dog.jpg')
df = pd.DataFrame(['Cat', 'Dog'], columns=['Image'])

print(cat.shape, dog.shape)

In [ ]:
fig = plt.figure(figsize = (8, 4))
ax1 = fig.add_subplot(1,2, 1)
ax1.imshow(cat)
ax2 = fig.add_subplot(1,2, 2)
ax2.imshow(dog)

In [ ]:
dog_r = dog.copy() # Red Channel
dog_r[:,:,1] = dog_r[:,:,2] = 0 # Set G,B pixels = 0
dog_g = dog.copy() # Green Channel
dog_g[:,:,0] = dog_r[:,:,2] = 0 # Set R,B, pixel = 0
dog_b = dog.copy() # Blue Channel
dog_b[:,:,0] = dog_b[:,:,1] = 0 # Set R,G pixel = 0

plot_image = np.concatenate((dog_r, dog_g, dog_b), axis=1)
plt.figure(figsize= (10,4))
plt.imshow(plot_image)

In [ ]:
dog_r[:,:,0]

In [ ]:
from skimage.color import rgb2gray

cgs = rgb2gray(cat)
dgs = rgb2gray(dog)

print('Image Shape:', cgs.shape, '\n')

# 2D pixel map
print('2D image pixel map')
print(np.round(cgs, 2), '\n')

# flattened pixel feature vector
print('Flattened pixel map:', (np.round(cgs.flatten(), 2)))

In [ ]:
fig = plt.figure(figsize = (8,4))
ax1 = fig.add_subplot(2,2, 1)
ax1.imshow(cgs, cmap="gray")
ax2 = fig.add_subplot(2,2, 2)
ax2.imshow(dgs, cmap="gray")
ax3 = fig.add_subplot(2,2, 3)
c_freq, c_bins, c_patches = ax3.hist(cgs.flatten(), bins=30)
ax4 = fig.add_subplot(2,2, 4)
d_freq, d_bins, d_patches = ax4.hist(dgs.flatten(), bins=30)


In [ ]:
from scipy.stats import describe

cat_rgb = cat.reshape((183*275), 3).T
dog_rgb = dog.reshape((194*259), 3).T

cs = describe(cat_rgb, axis=1)
ds = describe(dog_rgb, axis=1)

cat_rgb_range = cs.minmax[1] - cs.minmax[0]
dog_rgb_range = ds.minmax[1] - ds.minmax[0]
rgb_range_df = pd.DataFrame([cat_rgb_range, dog_rgb_range],
                            columns=['R_rnage', 'G_range', 'B_range'])
pd.concat([df, rgb_range_df], axis=1)


In [ ]:
cat_stats = np.array([np.round(cs.mean, 2), np.round(cs.variance, 2),
                      np.round(cs.kurtosis, 2), np.round(cs.skewness, 2), 
                      np.round(np.median(cat_rgb, axis=1), 2)]).flatten()
dog_stats = np.array([np.round(ds.mean, 2), np.round(ds.variance, 2),
                      np.round(ds.kurtosis, 2), np.round(ds.skewness, 2), 
                      np.round(np.median(dog_rgb, axis=1), 2)]).flatten()
stats_df = pd.DataFrame([cat_stats, dog_stats],
                        columns=['R_mean', 'G_mean', 'B_mean', 'R_var', 'G_var',
                                 'B_var', 'R_kurt', 'G_kurt', 'B_kurt', 'R_skew',
                                 'G_skew', 'B_skew', 'R_med', 'G_med', 'B_med'])
pd.concat([df, stats_df], axis=1)

In [ ]:
from skimage.feature import canny

cat_edges = canny(cgs, sigma=3)
dog_edges = canny(dgs, sigma=3)

fig = plt.figure(figsize=(8,4))
ax1 = fig.add_subplot(1,2, 1)
ax1.imshow(cat_edges, cmap='binary')
ax2 = fig.add_subplot(1,2, 2)
ax2.imshow(dog_edges, cmap='binary')

In [ ]:
from skimage.feature import hog
from skimage import exposure

fd_cat, cat_hog = hog(cgs, orientations=8, pixels_per_cell=(8,8),
                                            cells_per_block=(3,3), visualize=True)
fd_dog, dog_hog = hog(dgs, orientations=8, pixels_per_cell=(8,8),
                      cells_per_block=(3,3), visualize=True)

# rescaing intensity to get better plots
cat_hogs = exposure.rescale_intensity(cat_hog, in_range=(0, 0.04))
dog_hogs = exposure.rescale_intensity(dog_hog, in_range=(0, 0.04))

fig = plt.figure(figsize = (10,4))
ax1 = fig.add_subplot(1,2, 1)
ax1.imshow(cat_hogs, cmap='binary')
ax2 = fig.add_subplot(1,2, 2)
ax2.imshow(dog_hogs, cmap='binary')

In [ ]:
print(fd_cat, fd_cat.shape)

In [ ]:
from mahotas.features import surf
import mahotas as mh

cat_mh = mh.colors.rgb2gray(cat)
dog_mh = mh.colors.rgb2gray(dog)

cat_surf = surf.surf(cat_mh, nr_octaves=8, nr_scales=16, initial_step_size=1,
                     threshold=0.1, max_points=50)
dog_surf = surf.surf(dog_mh, nr_octaves=8, nr_scales=16, initial_step_size=1,
                     threshold=0.1, max_points=54)

fig = plt.figure(figsize= (10, 4))
ax1 = fig.add_subplot(1,2, 1)
ax1.imshow(surf.show_surf(cat_mh, cat_surf))
ax2 = fig.add_subplot(1,2, 2)
ax2.imshow(surf.show_surf(dog_mh, dog_surf))

In [ ]:
cat_surf_fds = surf.dense(cat_mh, spacing=10)
dog_surf_fds = surf.dense(dog_mh, spacing=10)
cat_surf_fds.shape

In [ ]:
!pip install mahotas

In [ ]:
from sklearn.cluster import KMeans

k = 20
km = KMeans(k, n_init=100, max_iter=100)

surf_fd_features = np.array([cat_surf_fds, dog_surf_fds])
km.fit(np.concatenate(surf_fd_features))

vbow_features = []
for feature_desc in surf_fd_features:
  labels = km.predict(feature_desc)
  vbow = np.bincount(labels, minlength=k)
  vbow_features.append(vbow)

vbow_df = pd.DataFrame(vbow_features)
pd.concat([df, vbow_df], axis=1)


In [ ]:
new_cat = io.imread('new_cat.jpg')
newcat_mh = mh.colors.rgb2gray(new_cat)
newcat_surf = surf.surf(newcat_mh, nr_octaves=8, nr_scales=16, initial_step_size=1,
                        threshold=0.1, max_points=50)

fig = plt.figure(figsize = (10, 4))
ax1 = fig.add_subplot(1,2, 1)
ax1.imshow(surf.show_surf(newcat_mh, newcat_surf))

In [ ]:
new_surf_fds = surf.dense(newcat_mh, spacing=10)
new_vbow = np.bincount(labels, minlength=k)
pd.DataFrame([new_vbow])

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity

eucdis = euclidean_distances(new_vbow.reshape(1,-1), vbow_features)
cossim = cosine_similarity(new_vbow.reshape(1,-1), vbow_features)

result_df = pd.DataFrame({'EuclideanDistance': eucdis[0],
                          'CosineSimilarity': cossim[0]})
pd.concat([df, result_df], axis=1)

